# Mentoría Ciencia de datos aplicada a la predicción de licencias médicas y comportamiento de los colaboradores

# Trabajo Práctico 1: Análisis y Visualización de Datos

Trabajar con el csv denominado data.csv y resolver las siguientes consignas:

1) ¿Cuáles son las columnas relevantes del dataset?
2) ¿Qué tipo de variable es cada una? Asegurar que tengan el tipo de datos adecuado. Por ejemplo, para el caso de las fechas usar libreria datetime para su conversión.
3) Exploren si hay valores faltantes y/o nulos en el dataset.
4) A partir de la fecha de nacimiento calcular la edad del colaborador que solicita la licencia. 

5) ¿Existen outliers en las variables seleccionadas?
6) Realicen un análisis estadístico de cada una de las variables numéricas: Cantidad de datos, mínimo, máximo, media, mediana, varianza, desviación estándar, cuartil 1, cuartil 3, rango intercuartílico.

    6.A) Elijan una o dos variables categóricas, repetir este análisis y sacar conclusiones.
    
    6.B) ¿El tiempo es una variable influyente en las distribuciones de estas variables? Ayudarse de gráficos para contestar esta pregunta. 
    
7) Representen gráficamente cada variable numérica eligiendo el gráfico que considere apropiado. Consideren la posibilidad de generar rangos de datos para su análisis y representación gráfica de las variables. 

    7.A) Repitan los gráficos agrupando por una o dos variables categóricas y sacar conclusiones.
    
8) Presenten una tabla de frecuencias y porcentajes para las variables categóricas estado civil y edad.
9) Realicen un gráfico para representar la tabla construida en el punto 7.
10) En todas las consignas saquen conclusiones de lo observado. 

## Carga del dataset

Se carga el dataset de solicitudes de licencias médicas.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None

url = 'https://raw.githubusercontent.com/MentoriaDiploDatos/Presentacion/main/data.csv'
df = pd.read_csv(url, sep=';')

## 1. Columnas relevantes

Se seleccionan las columnas del dataset relevantes para el siguiente planteo:

> ¿existe un sobredimensionamiento en la cantidad de días solicitados en las licencias médicas?

Asociado a este planteo, se presume que la dimensión temporal va a ser relevante por lo que también se incluirán columnas de fecha en el análisis.

### a) Exploración inicial del dataset

Se realiza una primer exploración del dataset para conocer su estructura:
 - dimensión (filas y columnas)
 - filas de ejemplo

In [ ]:
df.shape

In [ ]:
df.head(5)

### b) Selección de columnas relevantes

De las columnas listadas se seleccionan las siguientes:

In [ ]:
# se crea un dataset con las columnas relevantes
relevant_columns = ['fecha_creacion', 'dias_solicitados', 'fecha_inicio', 'fecha_fin', 'diagnostico',
                    'id_diagnostico', 'fecha_nacimiento', 'genero', 'tipo_licencia', 'estado_civil', 
                    'dias_aprobados', 'decision_medica', 'id_auditor', 'motivo_rechazo', 'categoria_diagnostico']
relev_df = df[relevant_columns]
relev_df.head()

## 2. Tipos de datos

Se analiza el tipo de datos de las columnas seleccionadas

In [ ]:
relev_df.dtypes

### a) Transformación de columnas de tipo fecha

Se comprueba que las variables de fecha tienen tipo *object*; se decide convertir dichas variables en tipo *datetime*.

In [ ]:
relev_df['fecha_creacion'] = pd.to_datetime(relev_df['fecha_creacion'])
relev_df['fecha_inicio'] = pd.to_datetime(relev_df['fecha_inicio'])
relev_df['fecha_fin'] = pd.to_datetime(relev_df['fecha_fin'])
relev_df['fecha_nacimiento'] = pd.to_datetime(relev_df['fecha_nacimiento'])

relev_df.dtypes

### b) Tipos de variables

Las variables seleccionadas se dividen en:

- **categóricas nominales**: *diagnostico*, *genero*, *tipo_licencia*, *estado_civil*, *decision_medica*, *id_auditor*, *motivo_rechazo*, *id_diagnostico*, *categoria_diagnostico*
- **numéricas discretas**: *fecha_creacion*, *dias_solicitados*, *fecha_inicio*, *fecha_fin*, *fecha_nacimiento*, *dias_aprobados*

> Nota: las variables de tipo fecha podrían ser consideradas tanto de tipo discreto como de tipo continuo, sin embargo, serán consideradas como discretas.  

## 3. Valores faltantes y/o nulos

Se explora el dataset para buscar **valores faltantes** (datos que no se encuentran presentes en una variable o columna determinada, ya sea porque no se registraron o porque se perdieron).

In [ ]:
# se identifican los valores nulos (None o NaN)
relev_df.isnull().sum()

In [ ]:
import missingno as msno

msno.bar(relev_df,figsize=(12, 6), sort="ascending",fontsize=12, color='orange')
msno.matrix(relev_df,figsize=(12, 6), fontsize=12, color=[0,0,0.2])
msno.heatmap(relev_df,figsize=(12, 6), fontsize=12)

El uso de la librería *missingno* nos revela valores faltantes en las siguientes columnas:

- fecha_fin
- decision_medica
- dias_aprobados
- id_auditor
- dias_solicitados
- id_diagnostico
- categoria_diagnostico
- motivo_rechazo

### a) Registros sin auditar

El análisis previo nos permite identificar un conjunto de variables con valores faltantes con una correlación total (corr=1). Estas variables son:

 - id_auditor
 - decision_medica
 - dias_aprobados

 Se observa que en el caso que un valor falta en una columna, falta en las otras; esto podría deberse a que los tres datos se completan cuando la solicitud es auditada; la ausencia de valores podría indicar que los registros no fueron auditados.

 La solución propuesta es eliminar las filas sin auditar, ya que no contribuyen a resolver la pregunta planteada inicialmente.

### b) Registros sin diagnóstico

También es posible identificar un segundo conjunto de variables con valores faltantes con una correlación alta (corr>0.7). Estas variables son:

 - id_diagnostico
 - categoria_diagnostico
 - motivo_rechazo

En este caso se observa que la ausencia de valores en estas columnas se ubica entre un 40% y un 60% de los valores.

Por el alto grado de correlación se descarta que se trate de valores faltantes completamenta al azar (MCAR). Se recomienda analizar con más detalle los valores faltantes para identificar posibles patrones.

### c) Calculo la cantidad de días de licencia que fueron  efectivamente autorizados

In [ ]:
# 4 CALCULO LA CANTIDAD DE DÍAS EFECTIVAMENTE AUTORIZADOS DE LICENCIA
def days_between(row):
    d1 = pd.to_datetime(row["fecha_inicio"], format="%Y-%m-%d")
    d2 = pd.to_datetime(row["fecha_fin"], format="%Y-%m-%d")
    return abs((d2 - d1).days)


relev_df["dias_lic_efectiva"] = relev_df.apply(days_between, axis=1)

## 4. Nueva columna edad

Se agrega al dataset una nueva columna edad tomando como dato base la fecha de nacimiento del solicitante. 

In [ ]:
def calculate_age(row):
    fecha_nacimiento = row['fecha_nacimiento']
    fecha_inicio = row['fecha_inicio']
    return fecha_inicio.year - fecha_nacimiento.year - ((fecha_inicio.month, fecha_inicio.day) < (fecha_nacimiento.month, fecha_nacimiento.day))

relev_df['edad'] = relev_df.apply(calculate_age, axis=1)
relev_df.head()

## 5. Outliers

Se buscan valores atípicos en las columnas seleccionadas mediante exploración visual.

### a) Variables numéricas de tipo fecha

Entre las variables numéricas se analizarán primeramente las variables de tipo fecha: *fecha_creacion*, *fecha_inicio*, *fecha_fin* y *fecha_nacimiento*

In [ ]:
pd.plotting.register_matplotlib_converters()

figure, axis = plt.subplots(2, 2, figsize=(12, 6))

axis[0, 0].hist(relev_df['fecha_creacion'], bins=24)
axis[0, 0].set_title("fecha_creacion")

axis[0, 1].hist(relev_df['fecha_nacimiento'], bins=24)
axis[0, 1].set_title("fecha_nacimiento")

axis[1, 0].hist(relev_df['fecha_inicio'], bins=24)
axis[1, 0].set_title("fecha_inicio")

axis[1, 1].hist(relev_df['fecha_fin'], bins=24)
axis[1, 1].set_title("fecha_fin")



#### i) fecha_creacion

Se conocía de antemano que el dataset correspondía a solicitudes realizadas durante un período de dos años, y esto se comprueba en el histograma.

> No se detectan valores atípicos

#### ii) fecha_nacimiento

Este campo debería reflejar la fecha de nacimiento de los solicitantes al momento de realizar la solicitud de licencia; si se considera que los solicitantes son trabajadores que cumplen con las leyes argentinas, el rango de edad de los solicitantes incluidos en el dataset deberían tener una edad entre 18 y 65 años, es decir una fecha de nacimiento posterior a enero de 1955 (65 años antes a la menor fecha de creación del dataset), y anterior a diciembre de 2004 (18 años antes a la mayor fecha de creación del dataset).

También se considera que en el punto 2.a) se realizó una transformación de la variable (conversión a tipo datetime), pero como también pudo comprobarse en el punto 3) no hay valores faltantes en la columna, por lo tanto no hay riesgo de que la transformación haya introducido valores atípicos.

Se comprueba observando valores mínimos y máximos y presentándolos de manera ordenada que hay valores atípicos.

> Se detectan valores atípicos anteriores a la fecha de nacimiento más antigua posible.

Se recomienda **investigar con el proveedor del dataset si existe alguna otro atributo en la base de datos (como por ejemplo el número de documento del solicitante) que permita inferir la fecha de nacimiento**. Si no es así, se recomienda **anular los valores para el dato *fecha_nacimiento* y luego imputarles un valor utilizando algún estadístico**.

In [ ]:
print("Valor mínimo: ", relev_df['fecha_nacimiento'].min(), " , Valor máximo: ", relev_df['fecha_nacimiento'].max())
relev_df['fecha_nacimiento'].sort_values()

#### iii) fecha_inicio

Este campo debería reflejar el inicio de las licencias médicas aprobadas; si se considera que las solicitudes del dataset tienen fecha de creación a partir de enero de 2021 y hasta diciembre de 2022, las fechas de inicio de licencias aprobadas deberían ser cercanas a las fechas de creación.
Se entiende que la fecha de inicio de una licencia puede ser anterior a la fecha de creación de la solicitud (es posible que una licencia médica se haya aprobado usando un canal de comunicación diferente al sistema de gestión de licencias, y que la carga de la misma se haya realizado a posteriori). Aún así, y dando un período de seguridad de seis meses entre el inicio de la licencia y el registro de la misma, se observan fechas de inicio que no corresponden a los valores esperados.

También se considera que en el punto 2.a) se realizó una transformación de la variable (conversión a tipo datetime), pero como también pudo comprobarse en el punto 3) no hay valores faltantes en la columna, por lo tanto no hay riesgo de que la transformación haya introducido valores atípicos.

Esto se comprueba observando valores mínimos y máximos y presentándolos de manera ordenada.

> Se detectan valores atípicos anteriores a la fecha de creación más antigua (aún considerando el período de seguridad de seis meses)

Se recomienda **investigar con el proveedor del dataset si las fechas de inicio de licencias detectadas como valores atípicos corresponden a casos reales o no**. 

In [ ]:
print("Valor mínimo: ", relev_df['fecha_inicio'].min(), " , Valor máximo: ", relev_df['fecha_inicio'].max())
relev_df['fecha_inicio'].sort_values()

#### iv) fecha_fin

Este campo debería reflejar el fin de las licencias médicas aprobadas. 

Se desconoce si existe un límite máximo en la cantidad de días de licencia que pueden otorgarse; también se desconoce si se registran solicitudes de licencias que ya hayan finalizado (fecha de creación mayor a la fecha de fin de la misma); aún así, y tal como se analizó para el dato *fecha_inicio*, se considera que si las solicitudes del dataset tienen fecha de creación a partir de enero de 2021 y hasta diciembre de 2022, entonces las fechas de fin de licencias aprobadas deberían ser cercanas a las fechas de creación.
Si se mantiene el supuesto del punto anterior que la fecha de inicio de una licencia puede ser anterior a la fecha de creación de la solicitud con un período de seguridad de seis meses entre el inicio de la licencia y el registro de la misma, entonces las fechas de finalización también deberían cumplir ese criterio.

También se considera que en el punto 2.a) se realizó una transformación de la variable (conversión a tipo datetime); pero, aunque hay un solo valor faltante en la columna como pudo verse en el punto 3), la conversión aplicada no modificó el valor nulo, por lo tanto no hay riesgo de que la transformación haya introducido valores atípicos.

No obstante, y a pesar de todas las consideraciones realizadas, se observan fechas de fin que no corresponden a los valores esperados.

Esto se comprueba observando valores mínimos y máximos y presentándolos de manera ordenada.

> Se detectan valores atípicos anteriores a la fecha de creación más antigua

Se recomienda **investigar con el proveedor del dataset si las fechas de inicio de licencias detectadas como valores atípicos corresponden a casos reales o no**. 

In [ ]:
print("Valor mínimo: ", relev_df['fecha_fin'].dropna().min(), " , Valor máximo: ", relev_df['fecha_fin'].dropna().max())
relev_df['fecha_fin'].dropna().sort_values()

### b) Variables numéricas de tipo entero

Las variables numéricas de tipo entero que se analizarán son: *dias_solicitados* y *dias_aprobados*. 

In [ ]:
sns.boxplot(relev_df[['dias_solicitados', 'dias_aprobados']], orient="h", palette="Set2")
sns.set(rc={'figure.figsize':(20,5)})

Las distribuciones de las dos variables están sesgadas a la derecha, y si bien hay valores alejados que analíticamente podrían considerarse como atípicos (distancias mayores a 3 veces el rango inter-cuartílico desde el percentil Q3), se desconoce si existe un límite máximo en la cantidad de días de licencia que pueden solicitarse y aprobararse.

> Se detectan valores atípicos (desde el punto de vista analítico)

Se recomienda **investigar con el proveedor del dataset si las cantidades de días solicitadas y aprobadas detectadas como valores atípicos corresponden a casos reales o no**. 

## 6. Descripción analítica de variables

Se realiza una descripción estadística de las siguientes variables numéricas y categóricas:
 - *fecha_creacion*
 - *fecha_nacimiento*
 - *fecha_inicio*
 - *fecha_fin*
 - *dias_solicitados*
 - *dias_aprobados*
 - *genero*
 - *diagnostico*


### a) fecha_creacion

In [ ]:
fecha_creacion_stats = pd.Series(relev_df['fecha_creacion'].to_numpy('datetime64'))
print("Count:  ", fecha_creacion_stats.count())
print("Min:    ", fecha_creacion_stats.min().strftime("%Y-%m-%d"))
print("Max:    ", fecha_creacion_stats.max().strftime("%Y-%m-%d"))
print("Median: ", fecha_creacion_stats.median().strftime("%Y-%m-%d"))
print("Mean:   ", fecha_creacion_stats.mean().strftime("%Y-%m-%d"))
print("Q1:     ", fecha_creacion_stats.quantile(0.25).strftime("%Y-%m-%d"))
print("Q3:     ", fecha_creacion_stats.quantile(0.75).strftime("%Y-%m-%d"))
print("IQR   : ", (fecha_creacion_stats.quantile(0.75) - fecha_creacion_stats.quantile(0.25)).days, "d")



### b) fecha_nacimiento

In [ ]:
fecha_nacimiento_stats = pd.Series(relev_df['fecha_nacimiento'].to_numpy('datetime64'))
print("Count:  ", fecha_nacimiento_stats.count())
print("Min:    ", fecha_nacimiento_stats.min().strftime("%Y-%m-%d"))
print("Max:    ", fecha_nacimiento_stats.max().strftime("%Y-%m-%d"))
print("Median: ", fecha_nacimiento_stats.median().strftime("%Y-%m-%d"))
print("Mean:   ", fecha_nacimiento_stats.mean().strftime("%Y-%m-%d"))
print("Q1:     ", fecha_nacimiento_stats.quantile(0.25).strftime("%Y-%m-%d"))
print("Q3:     ", fecha_nacimiento_stats.quantile(0.75).strftime("%Y-%m-%d"))
print("IQR   : ", (fecha_nacimiento_stats.quantile(0.75) - fecha_nacimiento_stats.quantile(0.25)).days, "d")

### c) fecha_inicio

In [ ]:
fecha_inicio_stats = pd.Series(relev_df['fecha_inicio'].to_numpy('datetime64'))
print("Count:  ", fecha_inicio_stats.count())
print("Min:    ", fecha_inicio_stats.min().strftime("%Y-%m-%d"))
print("Max:    ", fecha_inicio_stats.max().strftime("%Y-%m-%d"))
print("Median: ", fecha_inicio_stats.median().strftime("%Y-%m-%d"))
print("Mean:   ", fecha_inicio_stats.mean().strftime("%Y-%m-%d"))
print("Q1:     ", fecha_inicio_stats.quantile(0.25).strftime("%Y-%m-%d"))
print("Q3:     ", fecha_inicio_stats.quantile(0.75).strftime("%Y-%m-%d"))
print("IQR   : ", (fecha_inicio_stats.quantile(0.75) - fecha_inicio_stats.quantile(0.25)).days, "d")

### d) fecha_fin

In [ ]:
fecha_fin_stats = pd.Series(relev_df['fecha_fin'].dropna().to_numpy('datetime64'))
print("Count:  ", fecha_fin_stats.count())
print("Min:    ", fecha_fin_stats.min().strftime("%Y-%m-%d"))
print("Max:    ", fecha_fin_stats.max().strftime("%Y-%m-%d"))
print("Median: ", fecha_fin_stats.median().strftime("%Y-%m-%d"))
print("Mean:   ", fecha_fin_stats.mean().strftime("%Y-%m-%d"))
print("Q1:     ", fecha_fin_stats.quantile(0.25).strftime("%Y-%m-%d"))
print("Q3:     ", fecha_fin_stats.quantile(0.75).strftime("%Y-%m-%d"))
print("IQR   : ", (fecha_fin_stats.quantile(0.75) - fecha_fin_stats.quantile(0.25)).days, "d")

### e) dias_solicitados

In [ ]:
dias_solicitados_stats = pd.Series(relev_df['dias_solicitados'])
print("Count:  ", dias_solicitados_stats.count())
print("Min:    ", dias_solicitados_stats.min())
print("Max:    ", dias_solicitados_stats.max())
print("Median: ", dias_solicitados_stats.median())
print("Mean:   ", round(dias_solicitados_stats.mean(), 2))
print("Var:    ", round(dias_solicitados_stats.var(), 2))
print("Std:    ", round(dias_solicitados_stats.std(), 2))
print("Q1:     ", dias_solicitados_stats.quantile(0.25))
print("Q3:     ", dias_solicitados_stats.quantile(0.75))
print("IQR   : ", (dias_solicitados_stats.quantile(0.75) - dias_solicitados_stats.quantile(0.25)))

### f) dias_aprobados

In [ ]:
dias_aprobados_stats = pd.Series(relev_df['dias_aprobados'])
print("Count:  ", dias_aprobados_stats.count())
print("Min:    ", dias_aprobados_stats.min())
print("Max:    ", dias_aprobados_stats.max())
print("Median: ", dias_aprobados_stats.median())
print("Mean:   ", round(dias_aprobados_stats.mean(), 2))
print("Var:    ", round(dias_aprobados_stats.var(), 2))
print("Std:    ", round(dias_aprobados_stats.std(), 2))
print("Q1:     ", dias_aprobados_stats.quantile(0.25))
print("Q3:     ", dias_aprobados_stats.quantile(0.75))
print("IQR   : ", (dias_aprobados_stats.quantile(0.75) - dias_aprobados_stats.quantile(0.25)))

### g) genero

In [ ]:
relev_df['genero'].describe()

In [ ]:
relev_df['genero'].value_counts().plot(kind="bar",
                           title="Géneros",
                           rot=15,
                           xlabel="Género",
                           ylabel="Count")

### h) diagnostico

In [ ]:
relev_df['diagnostico'].describe()

In [ ]:
diag = relev_df.groupby(['diagnostico']).diagnostico.value_counts().nlargest(25)
diag.plot(kind="barh", title="Top 25 Diagnósticos")

In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

diag_df = pysqldf(""" SELECT STRFTIME("%Y-%m", fecha_creacion) as periodo, count(*) as casos
            FROM relev_df 
            WHERE diagnostico in ('CONTACTO ESTRECHO', 
                                'INFLUENZA DEBIDO A VIRUS DE LA INFLUENZA IDENTIFICADO (GRIPE)', 
                                'COVID SOSPECHOSO',
                                'COVID POSITIVO',
                                'GASTROENTERITIS VIRAL')
            GROUP BY STRFTIME("%Y-%m", fecha_creacion)
            ;""")
#sns.barplot(x=diag_df['periodos'], y=diag_df[[]], kind='bar')
diag_df.plot(x='periodo', y='casos', kind='bar', color=['skyblue'])

# 7) Representación gráfica de cada variable numérica
-  Eligiendo el gráfico que considere apropiado. Consideren la posibilidad de generar rangos de datos para su análisis y representación gráfica de las variables. 
- Repitan los gráficos agrupando por una o dos variables categóricas y sacar conclusiones.

In [ ]:
def filter_extreme_rows(df, column_list):
    # Iterate over the columns and filter out the rows with extreme values
    for column in column_list:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 3 * iqr
        upper_bound = q3 + 3 * iqr
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

    return df


# filtramos los valores extremos de las columnas numéricas
print(f"""shape antes de filtrar: {relev_df.shape}""")
relev_df_plot = filter_extreme_rows(
    relev_df, ["dias_solicitados", "dias_aprobados", "dias_lic_efectiva", "edad"]
)
print(f"""shape después de filtrar: {relev_df.shape}""")

In [ ]:
# Se crea una figura que contenga una cuadricula de 2 filas y 2 columnas para 4 gráficos
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# boxplot de dias_solicitados
sns.boxplot(ax=axes[0, 0], data=relev_df_plot, x="dias_solicitados", color="steelblue")

# boxplot de dias_aprobados
sns.boxplot(ax=axes[0, 1], data=relev_df_plot, x="dias_aprobados", color="steelblue")

# boxplot de dias_lic_efectiva
sns.boxplot(ax=axes[1, 0], data=relev_df_plot, x="dias_lic_efectiva", color="steelblue")

# histograma de edad
sns.histplot(ax=axes[1, 1], data=relev_df_plot, x="edad", color="steelblue")

In [ ]:
# Ahora repetimos los graficos pero agrupando por género
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# boxplot de dias_solicitados
sns.boxplot(
    ax=axes[0, 0], data=relev_df_plot, x="dias_solicitados", y="genero", color="steelblue"
)

# boxplot de dias_aprobados
sns.boxplot(
    ax=axes[0, 1], data=relev_df_plot, x="dias_aprobados", y="genero", color="steelblue"
)

# boxplot de dias_lic_efectiva
sns.boxplot(
    ax=axes[1, 0], data=relev_df_plot, x="dias_lic_efectiva", y="genero", color="steelblue"
)

# boxplot de edad
sns.histplot(ax=axes[1, 1], data=relev_df_plot, x="edad", hue="genero", color="steelblue")

# 8 Frecuencias absolutas y relativas de variables categóricas

Presenten una tabla de frecuencias y porcentajes para las variables categóricas estado civil y edad.

In [ ]:
# Frecuencias estado_civil
freq_abs_estado_civil = relev_df.estado_civil.value_counts()
freq_rel_estado_civil = relev_df.estado_civil.value_counts(normalize=True) * 100

# Se crea un dataframe con las frecuencias absolutas y relativas
df_freq_estado_civil = pd.DataFrame(
    {"freq_abs": freq_abs_estado_civil, "freq_rel": freq_rel_estado_civil}
)
df_freq_estado_civil

# Frecuencias genero
freq_abs_genero = relev_df.genero.value_counts()
freq_rel_genero = relev_df.genero.value_counts(normalize=True) * 100

# Se crea un dataframe con las frecuencias absolutas y relativas
df_freq_genero = pd.DataFrame(
    {"freq_abs": freq_abs_genero, "freq_rel": freq_rel_genero}
)
df_freq_genero

tabla_categoricas = (
    pd.crosstab(
        index=relev_df.estado_civil, columns=relev_df.genero, normalize=True, margins=True
    ).round(2)
    * 100
)
tabla_categoricas

# 9 Gráfica de frecuencias absolutas y relativas

In [ ]:
# Dropeamos registros donde All sea menor a 1 porciento
tabla_categoricas = tabla_categoricas[tabla_categoricas["All"] > 1]
# Dropeamos columna All y graficamos
tabla_categoricas.drop("All", axis=1, inplace=False).plot(
    kind="bar",
    figsize=(8, 6),
    rot=0,
    color=["steelblue", "orange", "green"],
    stacked=True,
)

# adjust plot size
plt.rcParams["figure.figsize"] = 14, 6
plt.show()